In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
from src import taskEncoder


In [3]:
import os 
datasets = os.listdir("datasets/desktop_cryptocurrency")
dpath = "datasets/desktop_cryptocurrency/{}"

In [4]:
tempDatasets = []
models = []
for data in datasets:
    temp = pd.read_csv(dpath.format(data))
    tempDatasets.append(temp)

numDatasets = len(tempDatasets)

In [17]:
train_label = tempDatasets[0][["label", "base_key"]]
tempDatasets[0] = tempDatasets[0].drop(["label"], axis=1)

datasetInputsSizes = []

tempHeaders = []
for index in range(len(tempDatasets)):
    data = tempDatasets[index]
    data = data.drop(["base_key"], axis=1)
    tempHeaders.append(list(data.columns))
    datasetInputsSizes.append(data.shape)

In [16]:
tempDatasets = []
models = []
tempHeaders = []
for data in datasets:
    temp = pd.read_csv(dpath.format(data))
    tempHeaders.append(list(temp.columns))
    tempDatasets.append(temp)

numDatasets = len(tempDatasets)
fullDataset = pd.concat(tempDatasets, axis=1)


In [7]:
uniqueUsers = fullDataset["user_id"].unique()
splitDatasets = []

for u in uniqueUsers:
    tempD = fullDataset[fullDataset["user_id"] == u]
    splitDatasets.append(tempD)


In [25]:
timeSeriesDatasets = {}
for data in splitDatasets:
    for i in range(1, numDatasets):
        if i not in timeSeriesDatasets:
            
            timeSeriesDatasets[i] = [torch.tensor(data[tempHeaders[i]].values, dtype=torch.float32)]
        else:
            timeSeriesDatasets[i].append(torch.tensor(data[tempHeaders[i]].values, dtype=torch.float32))


for i in range(1, numDatasets):
    d = timeSeriesDatasets[i]
    for j in range(len(d)):
        tempD = d[j]
        dshape = tempD.shape
        d[j] = tempD.reshape(1, dshape[0], dshape[1])

torch.Size([1, 75, 32])

In [176]:
for i in range(1, numDatasets):
    print(timeSeriesDatasets[i][2].shape)

torch.Size([1, 188, 37])
torch.Size([1, 188, 25])
torch.Size([1, 188, 16])
torch.Size([1, 188, 23])


In [26]:
encoders = []
for i in range(numDatasets):
    testMLP = taskEncoder.taskEncoder(dataType="tabular", dataDim = datasetInputsSizes[i][1])
    encoders.append(testMLP)

[2, 2]
[2, 2]
shape of the matrix:  2 2
input dimensions:  2 2
shape of the orthogonal matrix:  torch.Size([2, 2])
coefficients:  torch.Size([1])
shape of the matrix:  2 2
input dimensions:  2 2
shape of the orthogonal matrix:  torch.Size([2, 2])
coefficients:  torch.Size([1])
[32, 16]
[32, 16]
shape of the matrix:  32 32
input dimensions:  32 32
shape of the orthogonal matrix:  torch.Size([32, 32])
coefficients:  torch.Size([1])
shape of the matrix:  16 32
input dimensions:  16 32
shape of the orthogonal matrix:  torch.Size([32, 16])
coefficients:  torch.Size([1])
[20, 10]
[20, 10]
shape of the matrix:  20 20
input dimensions:  20 20
shape of the orthogonal matrix:  torch.Size([20, 20])
coefficients:  torch.Size([1])
shape of the matrix:  10 20
input dimensions:  10 20
shape of the orthogonal matrix:  torch.Size([20, 10])
coefficients:  torch.Size([1])
[11, 5]
[11, 5]
shape of the matrix:  11 11
input dimensions:  11 11
shape of the orthogonal matrix:  torch.Size([11, 11])
coefficient

In [34]:
testOutputs = []

for i in range(1, numDatasets):
    inputVals = tempDatasets[i].values
    testOutputs.append(encoders[i].getEmbeddings(torch.tensor(inputVals, dtype=torch.float32)))

In [ ]:
finalData = torch.concatenate(testOutputs, axis=1)
# use this to fit a model

In [21]:
rnnEncoders = []
for i in range(numDatasets):
    rnnEncoders.append(taskEncoder.taskEncoder(dataType="timeSeries", dataDim = datasetInputsSizes[i][1]))

generated matrix shape:  torch.Size([8, 2])
generated matrix shape:  torch.Size([8, 2])
generated matrix shape:  torch.Size([64, 32])
generated matrix shape:  torch.Size([64, 16])
generated matrix shape:  torch.Size([40, 20])
generated matrix shape:  torch.Size([40, 10])
generated matrix shape:  torch.Size([20, 11])
generated matrix shape:  torch.Size([20, 5])
generated matrix shape:  torch.Size([36, 18])
generated matrix shape:  torch.Size([36, 9])


In [32]:
finalFullDatasets = {}

for i in range(1, numDatasets):
    for dp in timeSeriesDatasets[i]:
        embedding = rnnEncoders[i].getEmbeddings(dp)
        if i not in finalFullDatasets:
            finalFullDatasets[i] = [embedding]
        else:
            finalFullDatasets[i].append(embedding)

In [45]:
fullDPs = []
for dp in range(len(finalFullDatasets)):
    tempDP = []
    for i in range(1, numDatasets):
        tempDP.append(finalFullDatasets[i][dp])

    fullDP = torch.concatenate(tempDP, axis=1)
    fullDPs.append(fullDP)